In [ ]:
import sys
sys.path.append("..") # for sibling import

import compyute as cp

# Example 3.1

### Deep neural network using sequential model

The goal of this model is to classify iris species based on numerical features.

### Step 1: Prepare data
You will need to download the dataset from https://www.kaggle.com/datasets/uciml/iris and place it into the *data* directory.

In [ ]:
# ! pip install pandas

In [ ]:
import pandas as pd

data_orig = pd.read_csv('../data/iris.csv')
data = data_orig.copy()
data.drop(columns=['Id'], inplace=True)
data.head()

In [ ]:
data.describe()

In [ ]:
data.info()

The labels are categorical values. To be used in the model, all data needs to be numerical.

In [ ]:
data_enc = data.copy()
data_enc["Species"] = data_enc["Species"].astype("category").cat.codes
data_enc.head()

Convert dataframe to a tensor.

In [ ]:
data_tensor = cp.Tensor(data_enc.to_numpy())
data_tensor.shape

Next the data is split into a training, validation and a testing dataset using the `split_train_test_val_data()` to evaluate the model later on. Before splitting the data is also shuffled, since sometimes raw data is sorted in some way.

In [ ]:
from compyute.preprocessing import split_train_val_test

train, val, test = split_train_val_test(data_tensor, ratio_val=0.2, ratio_test=0.2)
train[:5]

Features and labels are now seperated.

In [ ]:
X_train, y_train = train[:, :-1], train[:, -1].int()
X_val, y_val = val[:, :-1], val[:, -1].int()
X_test, y_test = test[:, :-1], test[:, -1].int()

In [ ]:
y_train[:10]

Neural networks tend to run into problems if values are very high. Therefore it is common to normalize the data. This can be done using the `normalize()` function, which applies min-max feature scaling to a tensor.<br><br>
$ X'=a+\frac{(X-X_{min})\cdot(b-a)}{X_{max}-X_{min}} $<br><br>, where<br><br>$ a $ ... lower bound<br>$ b $ ... upper bound

In [ ]:
from compyute.preprocessing import normalize

X_train = normalize(X_train, axis=0, l_bound=-1)
X_val = normalize(X_val, axis=0, l_bound=-1)
X_test = normalize(X_test, axis=0, l_bound=-1)
X_train[:5]

In [ ]:
print (f'{X_train.shape=}')
print (f'{y_train.shape=}')

print (f'{X_val.shape=}')
print (f'{y_val.shape=}')

print (f'{X_test.shape=}')
print (f'{y_test.shape=}')

### Step 2: Build the neural network structure
Here the individual layers of the neural network models are defined.

In [ ]:
import compyute.nn as nn
from compyute.nn.layers import *

model = nn.SequentialModel([
    Linear(4, 16),
    ReLU(),
    Batchnorm1d(16),
    Linear(16, 3),
])

The network is compiled to finalize the model. Besides the SGD optimizer, the framework also provides other algorithms like Adam. There are also multiple loss functions to choose from. Since this example explores a classification problem, the cross entropy loss function is used.

In [ ]:
model.compile(
    optimizer=nn.optimizers.SGD(),
    loss_fn=nn.losses.Crossentropy(),
    metric_fn=nn.metrics.accuracy
)

In [ ]:
from compyute.nn.analysis import model_summary
model_summary(model, (4,))

### Step 3: Train the model

In [ ]:
batch_size = 32

train_losses, train_scores, val_losses, val_scores = model.train(
    X_train, y_train, epochs=200, val_data=(X_val, y_val), verbose=1, batch_size=batch_size
)

In [ ]:
traces = {
    "train_losses" : train_losses,
    "train_scores" : train_scores
}

nn.analysis.plot_curve(traces=traces, figsize=(15, 3), title="train history", x_label="steps", y_label="loss/accuracy")

In [ ]:
traces = {
    "val_losses" : val_losses,
    "val_scores" : val_scores
}

nn.analysis.plot_curve(traces=traces, figsize=(15, 3), title="val history", x_label="steps", y_label="loss/accuracy")

### Step 4: Evaluate the model
Using the defined metric, the model's performance can be evaluated using testing/validation data.

In [ ]:
loss, accuracy = model.evaluate(X_test, y_test)
print(f'loss {loss:.4f}')
print(f'accuracy {100*accuracy:.2f}')